In [ ]:
!nvidia-smi

Tue Jan  9 01:59:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0              24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/proyectos3/
!ls

/content/drive/MyDrive/proyectos3
alexNet.ipynb  efficientNet.ipynb  models    PlantVillage_bgremoved  wandb
cnn.ipynb      GoogleNet.ipynb	   PlantDoc  resNet.ipynb


# Cargar Conjunto de Datos

In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import ConcatDataset


train_dir0 = './PlantVillage_bgremoved/train'
train_dir1 = './PlantDoc/train/'

val_dir0 = './PlantVillage_bgremoved/valid'
val_dir1 = './PlantDoc/test/'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset0 = ImageFolder(train_dir0, transform=transform,  is_valid_file=lambda x: x.endswith('.JPG'))
train_dataset1 = ImageFolder(train_dir1, transform=transform)

val_dataset0 = ImageFolder(val_dir0, transform=transform, is_valid_file=lambda x: x.endswith('.JPG'))
val_dataset1 = ImageFolder(val_dir1, transform=transform)

train_dataset = ConcatDataset([train_dataset0, train_dataset1])
val_dataset = ConcatDataset([val_dataset0, val_dataset1])

classes = train_dataset0.classes

classes

crear carga de datos para facilitar el acceso a los datos

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32
num_workers = 8 # Número de procesos que se utilizarán para cargar datos en paralelo
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch

def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [ ]:
device = get_default_device()

train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)

In [ ]:
device

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import precision_score, f1_score

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


class ImageClassificationBase(nn.Module):

    def training_step(self, batch):
        "calculate loss for a batch of training data"
        images, labels = batch
        out = self(images)                  # Generate predictions
        # nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        "calculate loss, accuracy, precision and f1 score for a batch of validation data"
        images, labels = batch
        out = self(images)                   # Generate prediction
        # nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss
        loss = F.cross_entropy(out, labels)  # Calculate loss
        preds = torch.argmax(out, dim=1)
        acc = accuracy(out, labels)          # Calculate accuracy

        # calculate precision and f1 score
        precision = precision_score(labels.cpu(), preds.cpu(), average='weighted', zero_division=1)
        f1 = f1_score(labels.cpu(), preds.cpu(), average='macro')
        return {'val_loss': loss.detach(), 'val_accuracy': acc, 'val_precision': precision, 'val_f1': f1}

    def validation_epoch_end(self, outputs):
        batch_losses = [x["val_loss"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()       # Combine loss
        batch_accuracy = [x["val_accuracy"] for x in outputs]
        epoch_accuracy = torch.stack(batch_accuracy).mean()
        return {"val_loss": epoch_loss.item(), "val_accuracy": epoch_accuracy.item()} # Combine accuracies

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_accuracy']))


In [ ]:
import torchvision.models as models

class AlexNet(ImageClassificationBase):
    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            # 1st conv layer
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2), #96
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            # 2nd conv layer
            nn.Conv2d(64, 192, kernel_size=5, padding=2), #256
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            # 3rd conv layer
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            # 4th conv layer
            nn.Conv2d(384, 256, kernel_size=3, padding=1), #384
            nn.ReLU(inplace=True),
            # 5th conv layer
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            # 1st fc layer
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            # 2nd fc layer
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            # output layer
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        # flatten the output of conv layers
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [ ]:
model = AlexNet(len(classes))
model = to_device(model, device)

In [ ]:
from torchsummary import summary              # for getting the summary of our model

# getting summary of the model
INPUT_SHAPE = (3, 224, 224)
print(summary(model.cuda(), (INPUT_SHAPE)))

In [ ]:
epochs = 20

max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:
!pip install wandb

In [ ]:
#import wandb

name = f'alexnet-{epochs}-epochs'
wandb.init(project="plantvillage-aug", name=name, config={
    "epochs": epochs,
    "learning_rate": max_lr,
    "grad_clip": grad_clip,
    "weight_decay": weight_decay,
    "opt_func": opt_func.__name__,
    "batch_size": batch_size,
    "dataset": "PlantVillage",
    "architecture": "AlexNet"
})

In [ ]:
# for training
@torch.no_grad()
def evaluate(model, val_loader):
    """Evaluates the model's performance on the validation set"""
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    # return model.validation_epoch_end(outputs)
    val_losses = [x["val_loss"] for x in outputs]
    epoch_loss = torch.stack(val_losses).mean()       # Combine loss
    val_accuracies = [x["val_accuracy"] for x in outputs]
    epoch_accuracy = torch.stack(val_accuracies).mean()
    val_precisions = [x["val_precision"] for x in outputs]
    epoch_precision = torch.tensor(sum(val_precisions)/len(val_precisions))
    val_f1s = [x["val_f1"] for x in outputs]
    epoch_f1 = torch.tensor(sum(val_f1s)/len(val_f1s))
    return {"val_loss": epoch_loss.item(), "val_accuracy": epoch_accuracy.item(), "val_precision": epoch_precision.item(), "val_f1": epoch_f1.item()} # Combine accuracies


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, weight_decay=0,
                grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # scheduler for one cycle learniing rate
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))


    for epoch in range(epochs):
        # Training
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # recording and updating learning rates
            lrs.append(get_lr(optimizer))
            sched.step()

            # logging to wandb
            wandb.log({"Train Loss": loss.item()})


        # validation
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)

        # logging to wandb
        wandb.log({
            "Epoch": epoch,
            "Val Loss": result['val_loss'],
            "Val Accuracy": result['val_accuracy'],
            "Val Precision": result['val_precision'],
            "Val F1": result['val_f1']
        })

    return history

In [ ]:
%%time
history = fit_one_cycle(
  epochs,
  max_lr,
  model,
  train_loader,
  val_loader,
  grad_clip=grad_clip,
  weight_decay=1e-4,
  opt_func=opt_func
)

In [ ]:
torch.save(model.state_dict(), f'models/plant_disease_aug_alex_{epochs}.pth')

In [ ]:
# clear nvidia cache
import torch, gc
gc.collect()
torch.cuda.empty_cache()